<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=73976adb39842b42b88ce64e86f77dc9cea9f54c73c61b57e53d0bad3856e003
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.2
    Uninstalling protobuf-6.33.2:
      Successfully uninstalled protobuf-6.33.2


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

  Attempting uninstall: yfinance
    Found existing installation: yfinance 1.0
    Uninstalling yfinance-1.0:
      Successfully uninstalled yfinance-1.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2026-01-07 13:23:19
-------------------
qualified stocks: 86
with latest results: 25
still star stocks: 14
-------------------
Initial Investment:  1.30 C
CY Investment:  1.57 C
Reserve:  1.68 K
Current:  1.44 C
-------------------
Today PnL: 4.47 K (0.03%)
Current PnL: -24.78 L (-15.8%)
CY Booked + Current PnL: -10.49 L (-6.69%)
-------------------
Total profit:  2.14 L
Total loss:  -26.92 L
-------------------
Total Booked + Current PnL: 16.49 L (12.7%)
Total Booked PnL: 41.27 L (31.79%)
Curr Year Booked PnL: 14.29 L (9.9%)
Prev Year Booked PnL: 26.98 L (20.78%)
Est FTT:  2.34 C
Est FTT PnL: 89.74 L (62.19%)
Deployed:  1.30 C
Current:  1.44 C
CAGR/XIRR %: 5.79%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
82,VOLTAS,1530.00,1.22,17.68,1.71,19.69,3859.0,33903.0,225645.0,6.24,73.0,X-MC,4.13,78.0,8.79,1.56,25.68,XY25,NTT,AC
51,NESTLEIND,1377.00,-0.64,15.62,4.99,21.40,15314.0,41465.0,306891.0,7.22,66.0,X-LC,5.23,12.0,2.71,2.13,22.94,XY25,NTT,FMCG
49,MASFIN,398.61,0.10,3.63,17.78,22.05,18053.0,3555.0,101535.0,-10.57,64.0,H-SC,6.97,164.0,0.20,0.70,47.59,XR,ATH,FINANCE
77,TTKPRESTIG,770.00,-1.52,-20.88,26.51,0.09,21137.0,-21044.0,79733.0,85.06,37.0,M-SC,3.21,253.0,-1.00,0.55,4.75,OX40N,NTT,DURABLES
17,COALINDIA,484.83,-0.00,5.02,13.53,19.23,21378.0,7548.0,158008.0,22.81,82.0,L-LC,6.25,179.0,0.35,1.10,27.36,XY25,ATH,MINING


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
72,TATAELXSI,9161.00,7.95,-16.84,58.77,32.03,64430.0,-22205.0,109630.0,-9.99,71.0,H-SC,7.02,148.0,-0.34,0.76,22.79,OX40N,NTT,IT
23,DMART,5391.45,4.78,-7.72,40.44,29.60,93149.0,-19260.0,230340.0,-21.32,51.0,X-LC,9.64,19.0,-0.21,1.60,12.78,X40N,ATH,FMCG
13,BSOFT,831.70,4.27,-20.36,89.32,50.79,100058.0,-28631.0,112022.0,2.05,57.0,H-SC,10.61,151.0,-0.29,0.78,30.43,XR,ATH,IT
44,JCHAC,2282.00,2.71,-35.32,54.60,-0.01,45133.0,-45143.0,82662.0,16813.04,47.0,M-SC,14.15,233.0,-1.00,0.57,5.80,OX40N,BTT,AC
50,MEDANTA,1486.00,2.38,3.34,20.06,24.08,26318.0,4246.0,131196.0,-5.01,60.0,X-SC,6.71,89.0,0.16,0.91,23.62,XY24,NTT,HEALTHCARE


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
56,RAJOOENG,143.1,-5.60,-28.44,95.09,39.61,69749.0,-29150.0,73350.0,-51.61,59.0,H-SC,31.70,135.0,-0.42,0.51,18.73,AR,ATH,MISC
75,TMPV,600.0,-2.14,-26.01,66.20,22.98,110649.0,-58755.0,167143.0,-23.08,48.0,X-LC,4.74,3.0,-0.53,1.16,5.13,XY24,NTT,AUTO
40,INDUSINDBK,1800.0,-1.93,-33.81,100.76,32.89,51494.0,-26100.0,51106.0,-731.43,64.0,L-MC,7.45,259.0,-0.51,0.35,40.74,XR,NTT,BANKS
76,TRIDENT,48.0,-1.88,-30.04,83.70,28.51,54021.0,-27714.0,64541.0,-17.17,31.0,M-SC,7.10,225.0,-0.51,0.45,10.44,XR,NTT,TEXTILES
64,SHALBY,327.0,-1.77,-26.53,70.27,25.10,93523.0,-48052.0,133091.0,879.27,37.0,M-SC,6.78,250.0,-0.51,0.92,6.66,XY24,NTT,HEALTHCARE


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,0.08,-0.89,109.77,107.91,160634.0,-1307.0,146337.0,-19.34,56.0,M-SC,10.15,216.0,-0.01,1.01,9.11,OX40N,ATH,CABLES


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,0.08,-0.89,109.77,107.91,160634.0,-1307.0,146337.0,-19.34,56.0,M-SC,10.15,216.0,-0.01,1.01,9.11,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,1.71,-13.98,16.28,0.02,24445.0,-24407.0,150152.0,128.79,56.0,M-SC,7.49,234.0,-1.00,1.04,30.61,OX40N,NTT,PAINTS
72,TATAELXSI,9161.00,7.95,-16.84,58.77,32.03,64430.0,-22205.0,109630.0,-9.99,71.0,H-SC,7.02,148.0,-0.34,0.76,22.79,OX40N,NTT,IT
47,KANSAINER,340.00,-0.52,-20.23,42.25,13.47,90884.0,-54558.0,215109.0,-67.15,57.0,H-SC,5.00,158.0,-0.60,1.49,9.38,XY24,NTT,PAINTS
77,TTKPRESTIG,770.00,-1.52,-20.88,26.51,0.09,21137.0,-21044.0,79733.0,85.06,37.0,M-SC,3.21,253.0,-1.00,0.55,4.75,OX40N,NTT,DURABLES


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
57,RECLTD,446.00,0.00,3.64,16.49,20.74,34723.0,7398.0,210568.0,35.83,74.0,M-MC,6.32,189.0,0.21,1.46,14.64,XY25,NTT,FINANCE
17,COALINDIA,484.83,-0.00,5.02,13.53,19.23,21378.0,7548.0,158008.0,22.81,82.0,L-LC,6.25,179.0,0.35,1.10,27.36,XY25,ATH,MINING
84,WIPRO,420.00,1.36,10.41,56.02,72.27,104810.0,17646.0,187094.0,-5.17,61.0,M-LC,6.05,99.0,0.17,1.30,17.23,XR,NTT,IT
49,MASFIN,398.61,0.10,3.63,17.78,22.05,18053.0,3555.0,101535.0,-10.57,64.0,H-SC,6.97,164.0,0.20,0.70,47.59,XR,ATH,FINANCE
37,IEX,219.00,1.31,2.22,45.40,48.63,92794.0,4437.0,204391.0,-31.97,68.0,H-SC,13.96,136.0,0.05,1.42,14.10,XR,NTT,MISC


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
49,MASFIN,398.61,0.10,3.63,17.78,22.05,18053.0,3555.0,101535.0,-10.57,64.0,H-SC,6.97,164.0,0.20,0.70,47.59,XR,ATH,FINANCE
84,WIPRO,420.00,1.36,10.41,56.02,72.27,104810.0,17646.0,187094.0,-5.17,61.0,M-LC,6.05,99.0,0.17,1.30,17.23,XR,NTT,IT
37,IEX,219.00,1.31,2.22,45.40,48.63,92794.0,4437.0,204391.0,-31.97,68.0,H-SC,13.96,136.0,0.05,1.42,14.10,XR,NTT,MISC
25,FINCABLES,1641.55,0.08,-0.89,109.77,107.91,160634.0,-1307.0,146337.0,-19.34,56.0,M-SC,10.15,216.0,-0.01,1.01,9.11,OX40N,ATH,CABLES
85,ZYDUSLIFE,1286.17,-0.45,-2.26,38.77,35.63,79414.0,-4740.0,204834.0,-15.85,52.0,H-MC,2.94,119.0,-0.06,1.42,14.03,AR,ATH,PHARMA


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
43,ITC,452.00,-0.63,-16.60,32.82,10.78,65783.0,-39881.0,200437.0,-50.14,13.0,X-LC,5.02,5.0,-0.61,1.39,0.00,X40,NTT,FMCG
22,DIXON,18931.72,-0.21,-18.87,61.80,31.26,115699.0,-43552.0,187216.0,-62.08,28.0,X-MC,18.19,56.0,-0.38,1.30,0.00,X40N,ATH,IT
7,AWL,485.00,-0.24,-25.38,110.65,57.19,269282.0,-82763.0,243364.0,-64.78,29.0,X-MC,8.42,58.0,-0.31,1.69,0.00,XY24,NTT,FMCG
52,PAGEIND,51605.08,0.11,-10.40,45.78,30.62,97237.0,-24654.0,212400.0,-36.35,33.0,X-MC,16.01,55.0,-0.25,1.47,0.11,X40,ATH,APPARELS
11,BATAINDIA,2096.00,-1.05,-39.63,123.94,35.20,96282.0,-50986.0,77684.0,1.81,36.0,X-SC,17.26,92.0,-0.53,0.54,0.00,X40,NTT,FOOTWEAR


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,UNITDSPR,1644.00,-0.33,0.79,19.83,20.77,46796.0,1840.0,235984.0,-7.91,38.0,X-MC,0.97,62.0,0.04,1.64,7.76,X40N,NTT,BREWERIES
65,SIEMENS,4671.50,-0.42,-16.31,49.97,25.51,77828.0,-30345.0,155750.0,-4.35,48.0,H-LC,1.42,49.0,-0.39,1.08,14.78,AR,ATH,ELECTRICAL
20,DABUR,735.00,0.16,1.90,41.02,43.70,103691.0,4714.0,252782.0,-6.50,66.0,X-MC,2.01,72.0,0.05,1.75,17.24,XY24,BTT,FMCG
81,VBL,671.64,0.63,1.40,33.79,35.67,108221.0,4434.0,320276.0,-10.61,67.0,X-LC,2.13,4.0,0.04,2.22,15.45,X40N,ATH,FMCG
45,JIOFIN,387.00,1.81,-2.23,27.32,24.48,71414.0,-5968.0,261397.0,-3.84,57.0,X-LC,2.29,36.0,-0.08,1.81,51.51,XY24,BTT,FINANCE


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
7,AWL,485.00,-0.24,-25.38,110.65,57.19,269282.0,-82763.0,243364.0,-64.78,29.0,X-MC,8.42,58.0,-0.31,1.69,0.00,XY24,NTT,FMCG
11,BATAINDIA,2096.00,-1.05,-39.63,123.94,35.20,96282.0,-50986.0,77684.0,1.81,36.0,X-SC,17.26,92.0,-0.53,0.54,0.00,X40,NTT,FOOTWEAR
22,DIXON,18931.72,-0.21,-18.87,61.80,31.26,115699.0,-43552.0,187216.0,-62.08,28.0,X-MC,18.19,56.0,-0.38,1.30,0.00,X40N,ATH,IT
43,ITC,452.00,-0.63,-16.60,32.82,10.78,65783.0,-39881.0,200437.0,-50.14,13.0,X-LC,5.02,5.0,-0.61,1.39,0.00,X40,NTT,FMCG
52,PAGEIND,51605.08,0.11,-10.40,45.78,30.62,97237.0,-24654.0,212400.0,-36.35,33.0,X-MC,16.01,55.0,-0.25,1.47,0.11,X40,ATH,APPARELS


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,QUESS,424.00,0.97,-25.61,92.01,42.84,44496.0,-16646.0,48360.0,-51.52,59.0,X-SC,28.51,83.0,-0.37,0.34,9.41,XY24,NTT,MISC
58,RELAXO,1176.00,-1.08,-48.20,194.00,52.31,145888.0,-69960.0,75200.0,-44.83,41.0,X-SC,7.62,91.0,-0.48,0.52,0.66,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,-1.05,-39.63,123.94,35.20,96282.0,-50986.0,77684.0,1.81,36.0,X-SC,17.26,92.0,-0.53,0.54,0.00,X40,NTT,FOOTWEAR
50,MEDANTA,1486.00,2.38,3.34,20.06,24.08,26318.0,4246.0,131196.0,-5.01,60.0,X-SC,6.71,89.0,0.16,0.91,23.62,XY24,NTT,HEALTHCARE
34,HONAUT,58357.33,-0.41,-16.41,71.46,43.32,97286.0,-26732.0,136140.0,-28.18,51.0,X-SC,7.78,90.0,-0.27,0.94,4.53,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4389.97,0.96,-9.62,33.55,20.71,114693.0,-36379.0,341858.0,-22.01,59.0,X-LC,2.67,1.0,-0.32,2.37,14.23,X40,ATH,IT
41,INFY,1972.00,0.36,9.75,22.32,34.24,76647.0,30511.0,343399.0,-14.57,50.0,X-LC,7.46,2.0,0.40,2.38,18.84,X40,NTT,IT
75,TMPV,600.00,-2.14,-26.01,66.20,22.98,110649.0,-58755.0,167143.0,-23.08,48.0,X-LC,4.74,3.0,-0.53,1.16,5.13,XY24,NTT,AUTO
81,VBL,671.64,0.63,1.40,33.79,35.67,108221.0,4434.0,320276.0,-10.61,67.0,X-LC,2.13,4.0,0.04,2.22,15.45,X40N,ATH,FMCG
43,ITC,452.00,-0.63,-16.60,32.82,10.78,65783.0,-39881.0,200437.0,-50.14,13.0,X-LC,5.02,5.0,-0.61,1.39,0.00,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
40,INDUSINDBK,1800.00,-1.93,-33.81,100.76,32.89,51494.0,-26100.0,51106.0,-731.43,64.0,L-MC,7.45,259.0,-0.51,0.35,40.74,XR,NTT,BANKS
49,MASFIN,398.61,0.10,3.63,17.78,22.05,18053.0,3555.0,101535.0,-10.57,64.0,H-SC,6.97,164.0,0.20,0.70,47.59,XR,ATH,FINANCE
13,BSOFT,831.70,4.27,-20.36,89.32,50.79,100058.0,-28631.0,112022.0,2.05,57.0,H-SC,10.61,151.0,-0.29,0.78,30.43,XR,ATH,IT
69,SURYODAY,216.00,-1.24,-18.83,51.58,23.03,74972.0,-33721.0,145350.0,57.89,54.0,H-SC,5.19,167.0,-0.45,1.01,44.22,XR,NTT,BANKS
39,INDIGOPNTS,1408.00,1.71,-13.98,16.28,0.02,24445.0,-24407.0,150152.0,128.79,56.0,M-SC,7.49,234.0,-1.00,1.04,30.61,OX40N,NTT,PAINTS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
0,5PAISA,593.00,-0.97,-26.85,54.35,12.90,86866.0,-58669.0,159827.0,135.58,70.0,H-SC,10.60,161.0,-0.68,1.11,32.90,OX40N,NTT,FINANCE
72,TATAELXSI,9161.00,7.95,-16.84,58.77,32.03,64430.0,-22205.0,109630.0,-9.99,71.0,H-SC,7.02,148.0,-0.34,0.76,22.79,OX40N,NTT,IT
82,VOLTAS,1530.00,1.22,17.68,1.71,19.69,3859.0,33903.0,225645.0,6.24,73.0,X-MC,4.13,78.0,8.79,1.56,25.68,XY25,NTT,AC
17,COALINDIA,484.83,-0.00,5.02,13.53,19.23,21378.0,7548.0,158008.0,22.81,82.0,L-LC,6.25,179.0,0.35,1.10,27.36,XY25,ATH,MINING
57,RECLTD,446.00,0.00,3.64,16.49,20.74,34723.0,7398.0,210568.0,35.83,74.0,M-MC,6.32,189.0,0.21,1.46,14.64,XY25,NTT,FINANCE


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.19
1,25,44.24
2,50,76.43


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    42.90
MC    30.76
LC    26.37
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     26.45
X40      23.77
X40N     12.77
XY25     10.71
XR        9.12
AR        8.64
OX40N     7.62
SR        0.95
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
X-MC    24.28
H-SC    23.00
X-LC    22.89
M-SC    11.24
X-SC     7.87
H-MC     4.67
M-MC     1.46
M-LC     1.30
L-LC     1.10
H-LC     1.08
L-SC     0.79
L-MC     0.35
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,16.99,-7.52,42.80
IT,12.79,-15.99,74.50
FINANCE,11.18,-10.49,56.87
MISC,7.20,-25.46,76.78
ELECTRICAL,5.93,-9.68,49.85
PAINTS,5.72,-11.86,28.48
INSURANCE,4.68,-0.36,34.71
PHARMA,3.97,-2.61,34.91
AUTO,2.79,-29.99,75.06


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3155113.0,21
AR,1326346.0,10
XR,1177998.0,12
X40,1091487.0,15
X40N,900834.0,9
OX40N,690955.0,10
XY25,348216.0,7
SR,283419.0,2


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3479487.0,24
X-MC,1481386.0,16
M-SC,1435634.0,15
X-LC,922009.0,13
X-SC,784701.0,8
H-MC,404510.0,3
L-SC,176408.0,2
M-LC,104810.0,1
H-LC,77828.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1231937.0      6
           AR         911364.0      5
M-SC       XY24       824409.0      6
H-SC       XR         753269.0      7
X-MC       X40        485207.0      7
           XY24       435740.0      3
X-LC       X40        415734.0      6
X-MC       X40N       391166.0      4
X-SC       X40N       308298.0      3
M-SC       OX40N      300969.0      5
H-SC       OX40N      299498.0      4
X-SC       XY24       285857.0      3
H-SC       SR         283419.0      2
H-MC       AR         209403.0      2
X-LC       X40N       201370.0      2
H-MC       XY24       195107.0      1
X-SC       X40        190546.0      2
M-SC       XR         182505.0      2
X-LC       XY24       182063.0      2
X-MC       XY25       169273.0      2
M-SC       AR         127751.0      2
X-LC       XY25       122842.0      3
M-LC       XR         104810.0      1
L-SC       OX40N       90488.0      1
           XR          85920.0      1
H-LC       AR          77828.0      1
L-MC       XR          51494.0      1
M-MC       XY25        34723.0      1
L-LC       XY25        21378.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 24.0 seconds
